# Landsat Quality Assessment Mask Generation using U-Net

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gee-community/ee-tensorflow-notebooks/blob/master/lc8_ee_qa_unet.ipynb)

This notebook provides a workflow to export training and validation data to build a deep learning model for QA masks with Landsat 8. The model will predict Cloud, Shadow, Snow, Water, Clear, and No Data classes in Landsat imagery.

## Setting up the environment

### Importing packages and checking versions

In [0]:
import os
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

# get numpy and matplotlib.pyplot
%pylab inline

In [0]:
# check to see what type of GPU is available
# best is Tesla P100-PCIE-16GB
!nvidia-smi -L

In [0]:
# Import, authenticate and initialize the Earth Engine library.
import ee
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [0]:
# Tensorflow setup.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K

print(tf.__version__)

In [0]:
# Folium setup.
import folium
print(folium.__version__)

### Declaring global variables that will be used throughout the workflow

In [0]:
# Specify cloud storage bucket to save data too
BUCKET = 'ee-rsqa'

# Specify names locations for outputs in Cloud Storage. 
FOLDER = 'training_data'
TRAINING_BASE = 'training_patches'
TESTING_BASE = 'testing_patches'
VAL_BASE = 'val_patches'

# Specify inputs (Landsat bands) to the model and the response variable.
opticalBands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
BANDS = opticalBands
RESPONSE = ['cloud','shadow','snow','water','land','nodata']
FEATURES = BANDS + RESPONSE



In [0]:
# specify a kernel/image size to use for the model
KERNEL_SIZE = 256

# create an EE kernel opject from the kernel size
list = ee.List.repeat(1, KERNEL_SIZE)
lists = ee.List.repeat(list, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

## Exporting training data to GCS

### Loading the Earth Engine assests
Here we use the Landsat 8 surface reflectance product and a hand labeled dataset, the [SPARCS dataset](https://www.usgs.gov/land-resources/nli/landsat/spatial-procedures-automated-removal-cloud-and-shadow-sparcs-validation).

In [0]:
# function to rescale LC8 data to 0-1 range
def rescale(img):
    return img.divide(10000).copyProperties(img).set('system:time_start',img.date())

# Use Landsat 8 surface reflectance data.
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').map(rescale)
# get image collection of hand labeled data and rename the bands
sparcs = ee.ImageCollection('projects/gmap/datasets/manual_qaMasks/sparcs_masks').select(['b1','b2','b3','b4','b5'],RESPONSE[:-1])

### Visualizing an example image

In [0]:
# grab the first image in the SPARCS image collection
maskImage = ee.Image(sparcs.first())
ndImage = ee.Image.constant(1).where(maskImage.reduce(ee.Reducer.sum()).eq(1),0).rename('nodata')
labelImage = ee.Image.cat([ndImage,maskImage])

# Filter the landsat collection that overlaps with the labeled image.
l8Image = l8sr.filterDate(maskImage.date().advance(-1,'hour'),maskImage.date().advance(1,'hour'))\
  .filterBounds(maskImage.geometry())\
  .mosaic()

# Use folium to visualize the imagery.
l8mapid = l8Image.getMapId({'bands': ['B7', 'B5', 'B3'], 'min': 0.05, 'max': 0.55, 'gamma':1.5})
qamapid = labelImage.getMapId({'bands': ['cloud','land','nodata'], 'max': 1})
label = ndImage.getMapId({'bands': ['nodata'], 'max': 1})

map = folium.Map(location=[-30.4853,-71.5639])
folium.TileLayer(
    tiles=l8mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='Landsat Image',
  ).add_to(map)

folium.TileLayer(
    tiles=qamapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='QA Mask',
  ).add_to(map)

map.add_child(folium.LayerControl())
map

### Exporting the training dataset
Here we loop through all of the labeled datasets, colocate with landsat imagery, and export the features as TFRecords to the cloud storage bucket we specified earlier.


In [0]:
# These numbers determined experimentally.
n = 10 # Number of shards in each polygon.
N = 150 # Total sample size in each image.

nImages = sparcs.size().getInfo() # total number of sparcs images

# convert label data image collection to list
sparcsList = sparcs.toList(nImages)

# Export all the training data (in many pieces), with one task 
# per geometry.
for i in range(nImages):
    # get the ith labeled image
    qaImage = ee.Image(sparcsList.get(i))

    # add a no data band
    # this is needed to avoid training issues with image edges where data will be 0 for all bands
    ndImage = ee.Image.constant(1).where(qaImage.reduce(ee.Reducer.sum()).gt(0),0).rename('nodata')
    labelImage = ee.Image.cat([ndImage,qaImage])
    
    # grab the landsat image that corresponds with the labeled data
    l8Image = l8sr.filterDate(qaImage.date().advance(-1,'hour'),qaImage.date().advance(1,'hour'))\
        .filterBounds(qaImage.geometry())\
        .mosaic()\
        .mask(ndImage.Not())

    # combine the labels and features
    exampleStack = ee.Image.cat([
        l8Image.select(BANDS),
        labelImage.select(RESPONSE)
    ]).float()
    # conver to a neighborhood array
    arrays = exampleStack.neighborhoodToArray(kernel)

    # sample n times within image and add to feature collection
    # n should be great enough to allow for image overlap
    # we oversample to flip, rotate, and augment the data during training
    geomSample = ee.FeatureCollection([])
    for j in range(n):
        sample = arrays.sample(
        region = qaImage.geometry(), 
        scale = 30, 
        numPixels = N / n, # Size of the shard.
        seed = j**2,
        tileScale = 8
        )
        geomSample = geomSample.merge(sample)

    # add random column to feature and split between training, test, and validataion
    geomSample = geomSample.randomColumn('random',i)
    training = geomSample.filter(ee.Filter.lt('random',0.6))
    testing = geomSample.filter(ee.Filter.rangeContains('random',0.6,0.84))
    validation = geomSample.filter(ee.Filter.gte('random',0.85))
  
    # set up the training export and start
    desc = TRAINING_BASE + '_i' + str(i)
    task = ee.batch.Export.table.toCloudStorage(
        collection = training,
        description = desc, 
        bucket = BUCKET, 
        fileNamePrefix = FOLDER + '/' + desc,
        fileFormat = 'TFRecord',
        selectors = FEATURES
    )
    task.start()

    # set up the testing export and start
    desc = TESTING_BASE + '_i' + str(i)
    task = ee.batch.Export.table.toCloudStorage(
        collection = testing,
        description = desc, 
        bucket = BUCKET, 
        fileNamePrefix = FOLDER + '/' + desc,
        fileFormat = 'TFRecord',
        selectors = FEATURES
    )
    task.start()

    # set up the validation export and start
    desc = VAL_BASE + '_i' + str(i)
    task = ee.batch.Export.table.toCloudStorage(
        collection = validation,
        description = desc, 
        bucket = BUCKET, 
        fileNamePrefix = FOLDER + '/' + desc,
        fileFormat = 'TFRecord',
        selectors = FEATURES
    )
    task.start()

## Building and training the Convolutional Neural Network (CNN)

This next section focuses on building and training a CNN that performs the predictions.

We will use the precreated CNN VGG-16 as the encoder branch of our network and create a custom decoder branch. 

### Defining and compiling the network

In [0]:
# custom decoder block to upsample the features in the network
# this specific decoder block uses a cov2d -> concat -> conv2d * n -> bilinear upsample
def decoder_block(input_tensor, concat_tensor=None, nFilters=512,nConvs=2,i=0,name_prefix="decoder_block"):
    deconv = input_tensor
    for j in range(nConvs):
        deconv = layers.Conv2D(nFilters, 3, activation='relu',
                               padding='same',name=f"{name_prefix}{i}_deconv{j+1}")(deconv)
        deconv = layers.BatchNormalization(name=f"{name_prefix}{i}_batchnorm{j+1}")(deconv)
        if j == 0:
            if concat_tensor is not None:
                 deconv = layers.concatenate([deconv,concat_tensor],name=f"{name_prefix}{i}_concat")
            deconv = layers.Dropout(0.2, seed=0+i,name=f"{name_prefix}{i}_dropout")(deconv)
    
    up = layers.UpSampling2D(interpolation='bilinear',name=f"{name_prefix}{i}_upsamp")(deconv)
    return up

In [0]:
# here we define the network using the VGG-16 encoder 
# and build our decoder from there

# specify an input tensor with an arbitrary shape for x and y dims
# has sample length channels as landsat bands we exported
inTensor = layers.Input(shape=[None,None,len(BANDS)],name="input")

# grab the vgg-16 encoder and build based off our input tensor
vgg16 = keras.applications.VGG19(include_top=False,weights=None,input_tensor=inTensor)

# grab the input and output tensors
base_in = vgg16.input
base_out = vgg16.output

# extract the tensors we will use to concatenate our decoders with
concat_layers = ["block5_conv3","block4_conv3","block3_conv3","block2_conv2","block1_conv2"]
concat_tensors = [vgg16.get_layer(layer).output for layer in concat_layers]

# define the decoder branch

decoder0 = decoder_block(base_out, nFilters=512,nConvs=1,i=0) # center block with no upsampling
decoder1 = decoder_block(decoder0, concat_tensor=concat_tensors[0], nFilters=512,nConvs=1,i=2) 
decoder2 = decoder_block(decoder1, concat_tensor=concat_tensors[1], nFilters=256,nConvs=1,i=3) 
decoder3 = decoder_block(decoder2, concat_tensor=concat_tensors[2], nFilters=128,nConvs=1,i=4) 
decoder4 = decoder_block(decoder3, concat_tensor=concat_tensors[3], nFilters=64,nConvs=1,i=5) 
# concat the final decoder block with the first encoder output
# drop out correlated connections in spatial space
outBranch = layers.concatenate([decoder4,concat_tensors[4]],name="out_block_concat1")
outBranch = layers.SpatialDropout2D(rate=0.2,seed=0,name="out_block_spatialdrop")(outBranch)

# perform some additional convolutions before predicting probabilites
outBranch = layers.Conv2D(64, 3, activation='relu', 
                          padding='same',name="out_block_conv1")(outBranch)
outBranch = layers.BatchNormalization(name="out_block_batchnorm1")(outBranch)
outBranch = layers.Conv2D(64, 3, activation='relu', 
                          padding='same',name="out_block_conv2")(outBranch)
outBranch = layers.BatchNormalization(name="out_block_batchnorm2")(outBranch)
# final convolution and softmax activation to get output probabilities
# nodes will equal the number of classes
outBranch = layers.Conv2D(len(RESPONSE), (1, 1),name='final_conv')(outBranch)
output = layers.Activation("softmax",name="final_out")(outBranch)
 
# declare our model with the inputs from the encoder and outputs from the decoder
model = models.Model(inputs=[base_in], outputs=[output],name="vgg16-unet")


In [0]:
# this is were we compile the model but first define some custom functions for
# metric monitoring and a custom loss function

# custom recall function
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

# custom precision function
def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

# custom F1-score function
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# soft dice loss function
# based on https://arxiv.org/pdf/1707.03237.pdf 
def dice_loss(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    true_sum = K.sum(K.square(y_true),-1) 
    pred_sum = K.sum(K.square(y_pred),-1)
    return 1 - ((2. * intersection + smooth) / (true_sum + pred_sum + smooth))

# define an adaptive learning rate based on training
lr_schedule = keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=500,
  decay_rate=1,
  staircase=False)

# compile the model
# uses Adam loss with adaptive learning rate
# soft dice loss as opjective function
# outputs accuracy, precision, recall, and f1
model.compile(optimizer=keras.optimizers.Adam(lr_schedule),
              loss=dice_loss,
              metrics=[keras.metrics.categorical_accuracy,
                       precision_m,
                       recall_m,
                       f1_m])

# display the model summary to see layers and parameters
model.summary()


### Defining training data pipeline



In [0]:
# Sizes of the training and evaluation datasets.
# based on sizes of exported data and spliting performed earlier
# ~80 total images with 150 samples per image = ~12000 samples
# ~65% are training, ~25% are testing, ~10% are validation
TRAIN_SIZE = 7800 
TEST_SIZE =  3000
VAL_SIZE = 1200 

# Specify model training parameters.
BATCH_SIZE = 20
EPOCHS = 20 
BUFFER_SIZE = 3000 # setting too large will give an Out of Memory (OOM) error

In [0]:
# Specify the size and shape of patches expected by the model.
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

In [0]:
def parse_tfrecord(example_proto):
    """The parsing function.
    Read a serialized example into the structure defined by FEATURES_DICT.
    Args:
    example_proto: a serialized Example.
    Returns: 
    A dictionary of tensors, keyed by feature name.
    """
    return tf.io.parse_single_example(example_proto, FEATURES_DICT)


def to_tuple(inputs):
    """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
    Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
    Args:
    inputs: A dictionary of tensors, keyed by feature name.
    Returns: 
    A dtuple of (inputs, outputs).
    """
    inputsList = [inputs.get(key) for key in FEATURES]
    stacked = tf.stack(inputsList, axis=0)
    # Convert from CHW to HWC
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked[:,:,:len(BANDS)], stacked[:,:,len(BANDS):]

# custom function to randomly augment the data during training
def transform(features,labels):
    x = tf.random.uniform(())
    # flip image on horizontal axis
    if x < 0.12: 
        feat = tf.image.flip_left_right(features)
        labl = tf.image.flip_left_right(labels)
    # flip image on vertical axis
    elif tf.math.logical_and(x >=0.12, x < 0.24):
        feat = tf.image.flip_up_down(features)
        labl = tf.image.flip_up_down(labels)
    # transpose image on bottom left corner
    elif tf.math.logical_and(x >=0.24, x < 0.36):
        feat = tf.image.flip_left_right(tf.image.flip_up_down(features))
        labl = tf.image.flip_left_right(tf.image.flip_up_down(labels))
    # rotate to the left 90 degrees
    elif tf.math.logical_and(x >=0.36, x < 0.48):
        feat = tf.image.rot90(features,k=1)
        labl = tf.image.rot90(labels,k=1)
    # rotate to the left 180 degrees
    elif tf.math.logical_and(x >=0.48, x < 0.60):
        feat = tf.image.rot90(features,k=2)
        labl = tf.image.rot90(labels,k=2)
    # rotate to the left 270 degrees
    elif tf.math.logical_and(x >=0.60, x < 0.72):
        feat = tf.image.rot90(features,k=3)
        labl = tf.image.rot90(labels,k=3)
    # transpose image on bottom right corner
    elif tf.math.logical_and(x >=0.72, x < 0.84):
        feat = tf.image.flip_left_right(tf.image.rot90(features,k=2))
        labl = tf.image.flip_left_right(tf.image.rot90(labels,k=2))
    else:
        feat = features
        labl = labels
    
    return feat,labl

def get_dataset(pattern,training=False):
    """Function to read, parse and format to tuple a set of input tfrecord files.
    Get all the files matching the pattern, parse and convert to tuple.
    Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
    Returns: 
    A tf.data.Dataset
    """
    glob = tf.gfile.Glob(pattern)
    dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
    dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
    dataset = dataset.map(to_tuple, num_parallel_calls=5)
    if training:
        dataset = dataset.map(transform)
    return dataset

In [0]:

def get_training_dataset():
    """Get the preprocessed training dataset
    Returns: 
    A tf.data.Dataset of training data.
    """
    glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + 't*'
    dataset = get_dataset(glob,training=True)
    dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
    return dataset

training = get_training_dataset()


In [0]:
def get_testing_dataset():
	"""Get the preprocessed evaluation dataset
  Returns: 
    A tf.data.Dataset of evaluation data.
  """
	glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + VAL_BASE + '*'
	dataset = get_dataset(glob)
	dataset = dataset.batch(1).repeat()
	return dataset

testing = get_testing_dataset()

## Training the model

In [0]:
# define a callback to stop training is the validation loss does not improve after 3 epochs
earlyStopping = callbacks.EarlyStopping(monitor='val_loss', patience=3, mode='min', restore_best_weights=True)

In [0]:
# train the model!!!
history = model.fit(x=training,
                    epochs=EPOCHS,
                    steps_per_epoch=(TRAIN_SIZE // BATCH_SIZE),
                    validation_data=testing,
                    validation_steps=TEST_SIZE,
                    callbacks=[earlyStopping],
                    initial_epoch=0,
                   )

Save the model to disc so we can use it later

In [0]:
# save the trained model to a file and move to your GCS bucket
model.save("vgg16unet_model.h5")
!gsutil cp vgg16unet_model.h5 gs://{BUCKET}/

### Displaying results of training

In [0]:
# plot the results of model training

fig, ax = plt.subplots(nrows=2, sharex=True, figsize=(10,5.5))

ax[0].plot(history.history['loss'],color='#1f77b4',label='Training Loss')
ax[0].plot(history.history['val_loss'],linestyle=':',marker='o',markersize=3,color='#1f77b4',label='Validation Loss')
ax[0].set_ylabel('Loss')
ax[0].set_ylim(0,0.15)
ax[0].legend()

ax[1].plot(history.history['categorical_accuracy'],color='#ff7f0e',label='Training Acc.')
ax[1].plot(history.history['val_categorical_accuracy'],linestyle=':',marker='o',markersize=3,color='#ff7f0e',label='Validation Acc.')
ax[1].set_ylabel('Accuracy')
ax[1].set_xlabel('Epoch')
ax[1].legend(loc="lower right")

ax[1].set_xticks(history.epoch)
ax[1].set_xticklabels(range(1,len(history.epoch)+1))
ax[1].set_xlabel('Epoch')
ax[1].set_ylim(0.8,1)

plt.legend()

# plt.savefig("/content/drive/My Drive/landsat_qa_samples/training.png",dpi=300,)

plt.show()


### Validating model results

In [0]:
def get_validation_dataset():
    """Get the preprocessed training dataset
    Returns: 
    A tf.data.Dataset of training data.
    """
    glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + VAL_BASE + '*'
    dataset = get_dataset(glob)
    dataset = dataset.batch(1)
    return dataset

validation = get_validation_dataset()

# evaluate the model
model.evaluate(validation)

## Deploying the trained model to AI Platform 

Here we save the model and deploy to AI platform so we can use it within Earth Engine

In [0]:
# save the model as a TF Estimator which is what 
MODEL_NAME = 'vgg16-unet'
TF_DIR = 'gs://{}/{}/'.format(BUCKET,MODEL_NAME)

# tf.keras.models.save_model(model,TF_DIR,save_format='tf')



In [0]:
from tensorflow.python.tools import saved_model_utils

meta_graph_def = saved_model_utils.get_meta_graph_def(TF_DIR, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None
for k,v in inputs.items():
    input_name = v.name
    break

output_name = None
for k,v in outputs.items():
    output_name = v.name
    break

# Make a dictionary that maps Earth Engine outputs and inputs to 
# AI Platform inputs and outputs, respectively.
import json
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: 'qa'}) + "'"
print(input_dict)
print(output_dict)

### Creating an EEified model

In [0]:
# Put the EEified model next to the trained model directory.
EEIFIED_DIR = 'gs://{}/eeified_{}/'.format(BUCKET,MODEL_NAME)
# change to your specific project
PROJECT = 'ee-sandbox'

# # You need to set the project before using the model prepare command.
!earthengine set_project {PROJECT}
!earthengine --no-use_cloud_api model prepare --source_dir {TF_DIR} --dest_dir {EEIFIED_DIR} --input {input_dict} --output {output_dict}


### Deploy to AI platform

In [0]:
import time

MODEL_NAME = 'lc8_qa_model'
VERSION_NAME = 'v' + str(int(time.time()))
print('Creating version: ' + VERSION_NAME)

!gcloud ai-platform versions create {VERSION_NAME} \
  --project {PROJECT} \
  --model {MODEL_NAME} \
  --origin {EEIFIED_DIR}/ \
  --runtime-version=1.14 \
  --framework "TENSORFLOW" \
  --python-version=3.5

In [0]:
!gcloud ai-platform versions describe $VERSION_NAME \
  --model $MODEL_NAME --project {PROJECT}

## Running Model on EE

In [0]:
# Use Landsat 8 SR data.
# Get image based on location and time range
geom = ee.Geometry.Point([-122.085,37.421]) # Google Campus
image = ee.Image(l8sr.filterBounds(geom)
  .filterDate("2019-06-01","2020-01-01")
  .first()).select(BANDS)

# Load the trained model and use it for prediction.
model = ee.Model.fromAiPlatformPredictor(**{
    'projectName': PROJECT,
    'modelName': MODEL_NAME,
    'version': VERSION_NAME,
    'inputTileSize': [144,144],
    'inputOverlapSize': [8,8],
    'inputShapes': ee.Dictionary({"array":[6]}),
    'proj': ee.Projection('EPSG:4326').atScale(30),
    'fixInputProj': True,
    'outputBands': {'qa': {
        'type': ee.PixelType.float(),
        'dimensions': 1
      }
    }
});

# run the predictions
predictions = model.predictImage(image.toFloat().toArray())

# find highest probability class
predClasses = predictions \
  .arrayArgmax() \
  .arrayFlatten([['qa']]);

# flatten probability array to image with bands  
predProbs = predictions \
  .arrayFlatten([['cloud','shadow','snow','water','land','nodata']]).toFloat()

# mask out the clear class
predClasses = predClasses.updateMask(predClasses.neq(4));



In [0]:
# Use folium to visualize the input imagery and the predictions.
# add Landsat 8 iamge to map
mapid = image.getMapId({'bands': ['B7', 'B5', 'B3'], 'min': 0.05, 'max': 0.55, 'gamma': 1.5})
map = folium.Map(location=[37.421,-122.085], zoom_start=12)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='Landsat Image',
  ).add_to(map)

# add predicted classes to map
mapid = predClasses.getMapId({'min':0,'max':5,'palette':'white,gray,cyan,blue,green,black','opacity':0.7})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='QA Classes',
  ).add_to(map)

# add class probabilities to map
mapid = predProbs.getMapId({'min':0,'max':1,'bands':"cloud,shadow,water",'opacity':0.5})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='Class Probabilities',
  ).add_to(map)


map.add_child(folium.LayerControl())
map